# Prompt Engineering

## Prompt Engineering Techniques

#### Zero-Shot Prompting
Zero-shot prompting involves providing the model with a task description without any examples. The model must generate the output based solely on the task description.

**Example:**
```
Translate the following English sentence to French: "Hello, how are you?"
```

#### Few-Shot Prompting
Few-shot prompting involves providing the model with a task description along with a few examples. This helps the model understand the task better and generate more accurate outputs.

**Example:**
```
Translate the following English sentences to French:
1. "Hello, how are you?" -> "Bonjour, comment ça va?"
2. "Good morning!" -> "Bon matin!"
3. "Thank you very much." -> "Merci beaucoup."
```

#### Chain-of-Thought Prompting
Chain-of-thought prompting involves providing the model with a task description and examples that include intermediate reasoning steps. This helps the model generate more coherent and logical outputs.

**Example:**
```
Solve the following math problem step-by-step:
Problem: If you have 3 apples and you buy 2 more, how many apples do you have in total?
1. Start with the initial number of apples: 3
2. Add the number of apples bought: 3 + 2
3. Calculate the total number of apples: 5
Answer: 5 apples
```

## Few-Shot Prompt Template

In [1]:
examples = [
    {
        "input": "Quem viveu mais, Ayrton Senna ou Santos Dumont?",
        "output": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quantos anos tinha Ayrton Senna quando morreu?
Resposta intermediária: Ayrton Senna tinha 34 anos quando morreu.
Pergunta de acompanhamento: Quantos anos tinha Santos Dumont quando morreu?
Resposta intermediária: Santos Dumont tinha 59 anos quando morreu.
Então a resposta final é: Santos Dumont
""",
    },
    {
        "input": "Quando nasceu o fundador da Embraer?",
        "output": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quem foi o fundador da Embraer?
Resposta intermediária: A Embraer foi fundada por Ozires Silva.
Pergunta de acompanhamento: Quando nasceu Ozires Silva?
Resposta intermediária: Ozires Silva nasceu em 8 de janeiro de 1931.
Então a resposta final é: 8 de janeiro de 1931
""",
    },
    {
        "input": "Quem foi o avô materno de Dom Pedro II?",
        "output": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quem foi a mãe de Dom Pedro II?
Resposta intermediária: A mãe de Dom Pedro II foi Maria Leopoldina de Áustria.
Pergunta de acompanhamento: Quem foi o pai de Maria Leopoldina de Áustria?
Resposta intermediária: O pai de Maria Leopoldina de Áustria foi Francisco II.
Então a resposta final é: Francisco II
""",
    },
    {
        "input": "Os diretores de 'Cidade de Deus' e 'Tropa de Elite' são do mesmo país?",
        "output": """
São necessárias perguntas de acompanhamento aqui: Sim.
Pergunta de acompanhamento: Quem é o diretor de 'Cidade de Deus'?
Resposta intermediária: O diretor de 'Cidade de Deus' é Fernando Meirelles.
Pergunta de acompanhamento: De onde é Fernando Meirelles?
Resposta intermediária: Brasil.
Pergunta de acompanhamento: Quem é o diretor de 'Tropa de Elite'?
Resposta intermediária: O diretor de 'Tropa de Elite' é José Padilha.
Pergunta de acompanhamento: De onde é José Padilha?
Resposta intermediária: Brasil.
Então a resposta final é: Sim
""",
    },
]

In [2]:
from langchain_core.prompts import ChatPromptTemplate

example_prompt = ChatPromptTemplate(
    [
        ('human', "{input}"),
        ('ai', "{output}")
    ]
)

In [3]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)

In [4]:
few_shot_prompt.invoke({}).to_messages()

[HumanMessage(content='Quem viveu mais, Ayrton Senna ou Santos Dumont?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Quantos anos tinha Ayrton Senna quando morreu?\nResposta intermediária: Ayrton Senna tinha 34 anos quando morreu.\nPergunta de acompanhamento: Quantos anos tinha Santos Dumont quando morreu?\nResposta intermediária: Santos Dumont tinha 59 anos quando morreu.\nEntão a resposta final é: Santos Dumont\n', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Quando nasceu o fundador da Embraer?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='\nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Quem foi o fundador da Embraer?\nResposta intermediária: A Embraer foi fundada por Ozires Silva.\nPergunta de acompanhamento: Quando nasceu Ozires Silva?\nResposta intermediária: Ozires Silva nasceu em 8 de janeiro

In [5]:
from langchain_core.prompts import ChatPromptTemplate

final_prompt = ChatPromptTemplate(
    [
        ('system', 'Você é um assistente virtual de perguntas e respostas. Você deve responder de acordo com o exemplo a seguir'),
        few_shot_prompt,
        ('human', '{input}')
    ]
)

In [ ]:
import os

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_SESSION_TOKEN'] = ''

In [13]:
import boto3

# Initialize the boto3 client for Bedrock
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name='us-west-2'
)

## Interact with the model with LangChain
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    client=bedrock_client,
    model='anthropic.claude-3-sonnet-20240229-v1:0',
    model_kwargs={'temperature': 0.9}
)

In [14]:
chain = final_prompt | llm

In [15]:
chain.invoke({"input": "Qual foi o time de Ariano Suassuna?"})

AIMessage(content='\nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Quem foi Ariano Suassuna?\nResposta intermediária: Ariano Suassuna foi um dramaturgo, romancista, poeta e ensaísta brasileiro.\nPergunta de acompanhamento: Ariano Suassuna era ligado a algum time de futebol?\nResposta intermediária: Não encontrei informações que indiquem que Ariano Suassuna era torcedor ou tinha vínculo com algum time de futebol específico. Ele era conhecido por sua obra literária e não por envolvimento com esportes.\nEntão a resposta final é: Ariano Suassuna não parece ter tido um "time" específico. Ele era um escritor e artista, não um esportista.', additional_kwargs={'usage': {'prompt_tokens': 727, 'completion_tokens': 216, 'total_tokens': 943}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-sonnet-20240229-v1:0'}, response_metadata={'usage': {'prompt_tokens': 727, 'completion_tokens': 216, 'total_tokens': 943}, 'stop_reason': 'end_turn', 'model_id': 

In [16]:
llm.invoke("Qual foi o time de Ariano Suassuna").content

"Ariano Suassuna foi um escritor, dramaturgo e poeta brasileiro. Ele não atuou profissionalmente em times de futebol. Suassuna é mais conhecido por suas obras literárias, como o Auto da Compadecida, O Romance d'A Pedra do Reino e o Príncipe do Sangue do Vai-e-Volta, inspiradas na cultura popular nordestina.\n\nSua obra resgatou e valorizou os folhetos de cordel, os romances populares, os desafios de viola, além de trazer elementos do Movimento Armorial, fundado por ele, que buscava resgatar as raízes eruditas da cultura popular brasileira.\n\nEmbora não tenha jogado futebol profissionalmente, Suassuna era um apaixonado pela cultura popular nordestina como um todo, incluindo suas manifestações esportivas. Mas sua contribuição maior se deu através de sua vasta produção literária."

In [17]:
chain.invoke({"input": "Qual foi o time de Ariano Suassuna?"}).content

'\nSão necessárias perguntas de acompanhamento aqui: Sim.\nPergunta de acompanhamento: Ariano Suassuna era jogador de futebol?\nResposta intermediária: Não, Ariano Suassuna não era jogador de futebol. Ele foi um escritor, dramaturgo e poeta brasileiro.\nPergunta de acompanhamento: Então ele não teve um time de futebol. Certo?\nResposta intermediária: Correto, Ariano Suassuna não teve um time de futebol, pois ele era um artista e escritor, não um jogador de futebol.\nEntão a resposta final é: Ariano Suassuna não teve um time de futebol, pois não era jogador.'